In [13]:
from multiprocess import Pool

file = open('data/19.txt', 'r')
data = file.read()

file.close()

In [14]:
import re
import random

RESOURCES = (1, 0.8, 0.5, 0.5)

class Factory:
    def __init__(self, blueprint):
        # Geode, Obsidian, Clay, Ore
        self.resources = [ 0, 0, 0, 0 ]

        self.costs = [
            [ 0, blueprint[5], 0, blueprint[4] ],
            [ 0, 0, blueprint[3], blueprint[2] ],
            [ 0, 0, 0,            blueprint[1] ],
            [ 0, 0, 0,            blueprint[0] ]
        ]

    def can_build(self, robot):
        cost = self.costs[robot]

        for i in range(4):
            if cost[i] > self.resources[i]:
                return False

        return True

    def reduce_resources(self, robot):
        cost = self.costs[robot]

        for i in range(4):
            self.resources[i] -= cost[i]

    def build(self):
        if self.resources[-1] == 0:
            return None

        # Fuck algorithms, let's be creative. 
        # Simulate harvest and decide randomly to build robots or not!
        for n, p in enumerate(RESOURCES):
            if self.can_build(n) and random.random() < p:
                self.reduce_resources(n)
                return n

        return None

In [15]:
blueprints = []

for line in data.splitlines():
    _, *blueprint = map(int, re.findall(r'\d+', line))
    blueprints.append(blueprint)

In [16]:
def harvest(factory: Factory, time: int):
    robots = [3]

    for minute in range(time):
        new = None

        if minute > 0 and minute < time - 1:
            new = factory.build()

        for robot in robots:
            factory.resources[robot] += 1

        if new != None:
            robots.append(new)

    return factory.resources[0]

def process_blueprint(blueprint, cycles: int, time: int):
    compute = lambda _: harvest(Factory(blueprint), time)
    
    with Pool(processes=16) as pool:
        return max(pool.map(compute, range(cycles)))        

In [17]:
geodes = [process_blueprint(b, 10000, 24) for b in blueprints]
answer = 0

for i, v in enumerate(geodes):
    answer += v * (i + 1)

answer

1599

In [19]:
geodes = [process_blueprint(b, 1000000, 32) for b in blueprints[:3]]
answer = 1

for _, v in enumerate(geodes):
    answer *= v

answer

14112